##Question Metadata
Domain: Molecular & Omics based Personalized Medicine

Subdomain: cfDNA based Diagnostics

Topic tags: cfDNA, end motifs, k-mers

Programming Language: Python

Tools: pysam, scipy, numpy

# TASK PROMPT

## Cell-free DNA Fragment End-Motif Profiling

### Problem Description

Cell-free DNA (cfDNA) whole genome sequencing (WGS) hg19-aligned BAM files from healthy and cancer samples are provided. The goal is to analyze cfDNA fragments and identify 4-nucleotide motifs at fragment ends that are significantly different between cancer and healthy samples. This analysis can reveal biological signals associated with cancer-specific fragmentation patterns.

---

### Task Requirements

The model must:

1. Extract 4-nucleotide motifs from both fragment ends of cfDNA reads.  
2. Focus on biologically relevant fragment lengths (e.g., 120–180 bp for nucleosomal DNA).  
3. Filter out low-quality reads or reads with ambiguous bases (N).  
4. Correctly handle strand orientation to reflect the actual DNA fragment.  
5. Compare motif frequencies between healthy and cancer samples.  
6. Compute statistical significance for each motif (e.g., Fisher’s exact test or chi-square).  
7. Account for multiple hypothesis testing.  
8. Rank motifs based on a combination of p-value, fold-change, and effect size.  
9. Return the top 10 motifs with structured output.  

---

### Input Format

The input will be a folder containing:

- `cancer.bam` : BAM file of cancer sample  
- `cancer.bam.bai` : BAM index file  
- `healthy.bam` : BAM file of healthy sample  
- `healthy.bam.bai` : BAM index file  

**Assumptions**: BAM files are already sorted, deduplicated, and aligned to hg19.

---

### Output Format

Return a list of dictionaries. Each dictionary must contain:

| Key | Type | Description |
|-----|------|-------------|
| motif | str | 4-nucleotide sequence (uppercase) |
| healthy_pct | float | Percentage in healthy sample (2 decimal places) |
| cancer_pct | float | Percentage in cancer sample (2 decimal places) |
| fold_change | float | cancer_pct / healthy_pct (handle zeros) |
| p_value | float | Statistical significance (< 0.05, 6 decimal places) |
| effect_size | float | Absolute difference |cancer_pct - healthy_pct| (2 decimal places) |

#### Example Output

```python
[
    {'motif': 'CCCA', 'healthy_pct': 0.85, 'cancer_pct': 2.14, 'fold_change': 2.52, 'p_value': 0.000100, 'effect_size': 1.29},
    {'motif': 'TGGG', 'healthy_pct': 1.92, 'cancer_pct': 0.63, 'fold_change': 0.33, 'p_value': 0.000300, 'effect_size': 1.29}
]


# Organize Data Files


In [26]:
# ===============================
# SETUP: create folders & install SRA toolkit
# ===============================
!mkdir -p /content/example_data
!mkdir -p /content/test_data
!apt-get update
!apt-get install -y sra-toolkit

# ===============================
# EXAMPLE DATA (first 10,000 reads)
# Healthy: SRR5946129 (~406 KB)
# Cancer:  SRR5946130 (~5.3 MB)
# ===============================

# Healthy example
!fasterq-dump SRR5946129 --split-files -O /content/example_data/
!head -n 40000 /content/example_data/SRR5946129_1.fastq > /content/example_data/healthy_R1.fastq
!head -n 40000 /content/example_data/SRR5946129_2.fastq > /content/example_data/healthy_R2.fastq

# Cancer example
!fasterq-dump SRR5946130 --split-files -O /content/example_data/
!head -n 40000 /content/example_data/SRR5946130_1.fastq > /content/example_data/cancer_R1.fastq
!head -n 40000 /content/example_data/SRR5946130_2.fastq > /content/example_data/cancer_R2.fastq

# Optional: remove original large FASTQ to save space
!rm /content/example_data/SRR5946129_*.fastq
!rm /content/example_data/SRR5946130_*.fastq

# Verify example_data
!ls -lh /content/example_data/

# ===============================
# TEST DATA (first 10,000 reads)
# Healthy: SRR5946131 (~3.2 MB)
# Cancer:  SRR5946132 (~6.7 MB)
# ===============================

# Healthy test
!fasterq-dump SRR5946131 --split-files -O /content/test_data/
!head -n 40000 /content/test_data/SRR5946131_1.fastq > /content/test_data/healthy_test_R1.fastq
!head -n 40000 /content/test_data/SRR5946131_2.fastq > /content/test_data/healthy_test_R2.fastq

# Cancer test
!fasterq-dump SRR5946132 --split-files -O /content/test_data/
!head -n 40000 /content/test_data/SRR5946132_1.fastq > /content/test_data/cancer_test_R1.fastq
!head -n 40000 /content/test_data/SRR5946132_2.fastq > /content/test_data/cancer_test_R2.fastq

# Optional: remove original large FASTQ to save space
!rm /content/test_data/SRR5946131_*.fastq
!rm /content/test_data/SRR5946132_*.fastq

# Verify test_data
!ls -lh /content/test_data/


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 3,917 B in 1s (3,754 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sra-toolkit is already the newest version (2.11.3+

# Expert Solution

In [25]:
# ===============================
# SETUP: create folders & install SRA toolkit
# ===============================
!mkdir -p /content/example_data
!mkdir -p /content/test_data
!apt-get update
!apt-get install -y sra-toolkit

# ===============================
# EXAMPLE DATA
# ===============================
# Healthy example
!fasterq-dump SRR5946129 -O /content/example_data/
# Cancer example
!fasterq-dump SRR5946130 -O /content/example_data/

# Rename for clarity
!mv /content/example_data/SRR5946129.fastq /content/example_data/healthy_R1.fastq
!mv /content/example_data/SRR5946130.fastq /content/example_data/cancer_R1.fastq

# ===============================
# TEST DATA
# ===============================
# Healthy test
!fasterq-dump SRR5946131 -O /content/test_data/
# Cancer test
!fasterq-dump SRR5946132 -O /content/test_data/

# Rename for clarity
!mv /content/test_data/SRR5946131.fastq /content/test_data/healthy_test_R1.fastq
!mv /content/test_data/SRR5946132.fastq /content/test_data/cancer_test_R1.fastq

# ===============================
# VERIFY ALL FILES
# ===============================
print("Example dataset files:")
!ls -lh /content/example_data/
print("Test dataset files:")
!ls -lh /content/test_data/


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (4,180 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sra-toolkit is already the newest version (2.11.3+

In [22]:
!rm -f /content/example_data/*.fastq
!rm -f /content/test_data/*.fastq


#Test Cases

In [30]:
# Print the first 5 reads of cancer test FASTQ
# Each read = 4 lines (header, sequence, plus, quality)
# Using the original SRR filename for the FASTQ
# 5 reads x 4 lines = 20 lines
!head -n 20 /content/test_data/SRR5946132.fastq


@SRR5946132.1 1 length=292
GTGCCAGCAGCCGCGGTAATACGTATGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTGATTAAGTCTGATGTGAAAGCCCGGAGCTCAACTCCGGAATGGCATTGGAAACTGGTTAACTTGAGTGCAGTAGAGGTAAGTGGAACTCCATGTGTAGCGGTGGAATGCGTAGATATATGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGTGTGGGTAGCAAACAGGATTAGATACCCCCGTAGTCC
+SRR5946132.1 1 length=292
ICICIGIIIIIIGIIGIIIGIGIIFIIIIIIIIIEIIIIIIIIIIIGIIIIIIIIIGIIIIIIIIII9IFIII=HIIIGIIII>AIIB?IEIIIIIIIIIIIIII7IIGIIIIIIIIIIIIGCIICIIIIIIIIIIIIIIIIIIIIFIIIICFIIIFIIIIIIIIIIIG,,IIIIIIIIIIIIIIIDIIIIIIIIIIIIGIIIIIIIIIG:IIII*4IIIIIIIIID6IIEIIIII6I<IIIIIIIIIIIIIIIIIGIIIIIIIIIIIIIIIIIIIIIIIIIIIIIGCICII
@SRR5946132.2 2 length=292
GTGCCAGCAGCCGCGGTAATACGTATGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTGATTAAGTCTGATGTGAAAGCCCGGAGCTCAACTCCGGAATGGCATTGGAAACTGGTTAACTTGAGTGCAGTAGAGGTAAGTGGAACTCCATGTGTAGCGGTGGAATGCGTAGATATATGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGTGTGGGTAGCAAACAGGATTAGATACCCTGGTAGTCC
+SRR5946132.2 2 length=292
ICCIIAFFGGIGG

In [31]:
# Print the first 5 reads of healthy test FASTQ
# Using the original SRR filename for the FASTQ
!head -n 20 /content/test_data/SRR5946131.fastq


@SRR5946131.1 1 length=292
GTGTCAGCAGCCGCGGTAATACGTATGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTGATTAAGTCTGATGTGAAAGCCCGGAGCTCAACTCCGGAATGGCATTGGAAACTGGTTAACTTGAGTGCAGTAGAGGTAAGTGGAACTCCATGTGTTGCGGTGGAATGCGTAGATATATGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGTGTGGGTAGCAAACAGGATTAGAAACCCCCGTAGTCA
+SRR5946131.1 1 length=292
9C@BCIGIGGGIGCHEGFIIFIIGDIIEIICECGIGGICDIIIGGGHFIIIGIGIIIEI2=3IICGIIIIIIIIIIIIGIIIIIFIGE:IIIIIIFIH7IHIIIE<IIIIIIII4@IIIHGCIIIIFIII8IIDHIIIIIIIIIIIIIII6>8>II,,IHI6@IIIIIFF9I6IIIIC<III=IEIIIIIIIIIIIIIFIIIIIIF5:IC@6B=E?/@IIIIII,II=IIIIIII6II,CIIIIIIIIFIGGIGIFII@II<EEIIIIIIIIE6IFIIIIIIIIIIIII4B-
@SRR5946131.2 2 length=292
GTGTCAGCCGCCGCGGTAATACGTATGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTGATTAAGTCTGATGTGAAAGCCCGGAGCTCAACTCCGGAATGGCATTGGAAACTGGTTAACTTGAGTGCAGTAGAGGTAAGTGGAACTCCATGTGTAGCGGTGGAATGCGTAGATATATGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGTGTGGGTAGCAAACAGGATTAGATACCCTTGTAGTCC
+SRR5946131.2 2 length=292
CCCCCIIIIIGII

In [32]:
# Verify that the healthy test file SRR5946131.fastq is smaller than 11 MB
# Lists the file size in human-readable format
!ls -lh /content/test_data/SRR5946131.fastq


-rw-r--r-- 1 root root 3.2M Feb 17 03:10 /content/test_data/SRR5946131.fastq


In [33]:
# Print the first 10 reads from SRR5946132.fastq
# Each read = 4 lines, so 10 reads x 4 lines = 40 lines
!head -n 40 /content/test_data/SRR5946132.fastq


@SRR5946132.1 1 length=292
GTGCCAGCAGCCGCGGTAATACGTATGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTGATTAAGTCTGATGTGAAAGCCCGGAGCTCAACTCCGGAATGGCATTGGAAACTGGTTAACTTGAGTGCAGTAGAGGTAAGTGGAACTCCATGTGTAGCGGTGGAATGCGTAGATATATGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGTGTGGGTAGCAAACAGGATTAGATACCCCCGTAGTCC
+SRR5946132.1 1 length=292
ICICIGIIIIIIGIIGIIIGIGIIFIIIIIIIIIEIIIIIIIIIIIGIIIIIIIIIGIIIIIIIIII9IFIII=HIIIGIIII>AIIB?IEIIIIIIIIIIIIII7IIGIIIIIIIIIIIIGCIICIIIIIIIIIIIIIIIIIIIIFIIIICFIIIFIIIIIIIIIIIG,,IIIIIIIIIIIIIIIDIIIIIIIIIIIIGIIIIIIIIIG:IIII*4IIIIIIIIID6IIEIIIII6I<IIIIIIIIIIIIIIIIIGIIIIIIIIIIIIIIIIIIIIIIIIIIIIIGCICII
@SRR5946132.2 2 length=292
GTGCCAGCAGCCGCGGTAATACGTATGTCCCGAGCGTTATCCGGATTTATTGGGCGTAAAGCGAGCGCAGACGGTTGATTAAGTCTGATGTGAAAGCCCGGAGCTCAACTCCGGAATGGCATTGGAAACTGGTTAACTTGAGTGCAGTAGAGGTAAGTGGAACTCCATGTGTAGCGGTGGAATGCGTAGATATATGGAAGAACACCAGTGGCGAAGGCGGCTTACTGGACTGTAACTGACGTTGAGGCTCGAAAGTGTGGGTAGCAAACAGGATTAGATACCCTGGTAGTCC
+SRR5946132.2 2 length=292
ICCIIAFFGGIGG

In [34]:
# Calculate total number of nucleotide bases in SRR5946131.fastq
# Uses awk to sum the length of all sequence lines (every 2nd line in FASTQ format)
# NR%4==2 selects the sequence lines
!awk 'NR%4==2 {total += length($0)} END {print "Total bases:", total}' /content/test_data/SRR5946131.fastq


Total bases: 1464670


# Download files

In [35]:
# Zip example_data, test_data, and your notebook into project_data.zip
!zip -r project_data.zip example_data test_data "Molecular & Omics based Personalized Medicine task testing template.ipynb"


	zip warning: name not matched: Molecular & Omics based Personalized Medicine task testing template.ipynb
  adding: example_data/ (stored 0%)
  adding: example_data/cancer_R1.fastq (stored 0%)
  adding: example_data/SRR5946129.fastq (deflated 89%)
  adding: example_data/SRR5946130.fastq (deflated 88%)
  adding: example_data/healthy_R2.fastq (stored 0%)
  adding: example_data/cancer_R2.fastq (stored 0%)
  adding: example_data/healthy_R1.fastq (stored 0%)
  adding: test_data/ (stored 0%)
  adding: test_data/healthy_test_R1.fastq (stored 0%)
  adding: test_data/cancer_test_R2.fastq (stored 0%)
  adding: test_data/SRR5946132.fastq (deflated 90%)
  adding: test_data/SRR5946131.fastq (deflated 89%)
  adding: test_data/cancer_test_R1.fastq (stored 0%)
  adding: test_data/healthy_test_R2.fastq (stored 0%)


In [36]:
!ls


example_data			fasterq.tmp.cb550c753c5b.36309	SRR2129993
fasterq.tmp.cb550c753c5b.11391	fasterq.tmp.cb550c753c5b.44125	SRR3179790
fasterq.tmp.cb550c753c5b.19812	fasterq.tmp.cb550c753c5b.4537	SRR33483210
fasterq.tmp.cb550c753c5b.22407	fasterq.tmp.cb550c753c5b.5951	SRR33483279
fasterq.tmp.cb550c753c5b.23535	project_data.zip		SRR5875050
fasterq.tmp.cb550c753c5b.27625	sample_data			test_data


In [39]:
from google.colab import files
files.download("project_data.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
!unzip -l project_data.zip


Archive:  project_data.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2026-02-17 03:10   example_data/
        0  2026-02-17 03:10   example_data/cancer_R1.fastq
   415650  2026-02-17 03:10   example_data/SRR5946129.fastq
  5525474  2026-02-17 03:10   example_data/SRR5946130.fastq
        0  2026-02-17 03:10   example_data/healthy_R2.fastq
        0  2026-02-17 03:10   example_data/cancer_R2.fastq
        0  2026-02-17 03:10   example_data/healthy_R1.fastq
        0  2026-02-17 03:10   test_data/
        0  2026-02-17 03:10   test_data/healthy_test_R1.fastq
        0  2026-02-17 03:10   test_data/cancer_test_R2.fastq
  6991462  2026-02-17 03:10   test_data/SRR5946132.fastq
  3266204  2026-02-17 03:10   test_data/SRR5946131.fastq
        0  2026-02-17 03:10   test_data/cancer_test_R1.fastq
        0  2026-02-17 03:10   test_data/healthy_test_R2.fastq
---------                     -------
 16198790                     14 files


In [41]:
!zip -r project_data.zip example_data/ test_data/ "Molecular & Omics based Personalized Medicine task testing template.ipynb"


	zip warning: name not matched: Molecular & Omics based Personalized Medicine task testing template.ipynb
updating: example_data/ (stored 0%)
updating: example_data/cancer_R1.fastq (stored 0%)
updating: example_data/SRR5946129.fastq (deflated 89%)
updating: example_data/SRR5946130.fastq (deflated 88%)
updating: example_data/healthy_R2.fastq (stored 0%)
updating: example_data/cancer_R2.fastq (stored 0%)
updating: example_data/healthy_R1.fastq (stored 0%)
updating: test_data/ (stored 0%)
updating: test_data/healthy_test_R1.fastq (stored 0%)
updating: test_data/cancer_test_R2.fastq (stored 0%)
updating: test_data/SRR5946132.fastq (deflated 90%)
updating: test_data/SRR5946131.fastq (deflated 89%)
updating: test_data/cancer_test_R1.fastq (stored 0%)
updating: test_data/healthy_test_R2.fastq (stored 0%)


In [42]:
# Step 1: Remove all 0-byte FASTQ files from example_data and test_data
!find example_data/ -type f -size 0 -name "*.fastq" -delete
!find test_data/ -type f -size 0 -name "*.fastq" -delete

# Step 2: Zip everything together (example_data, test_data, and your notebook)
!zip -r project_data.zip "Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb" example_data/ test_data/

# Step 3: List contents to verify
!unzip -l project_data.zip


	zip warning: name not matched: Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb
updating: example_data/ (stored 0%)
updating: example_data/SRR5946129.fastq (deflated 89%)
updating: example_data/SRR5946130.fastq (deflated 88%)
updating: test_data/ (stored 0%)
updating: test_data/SRR5946132.fastq (deflated 90%)
updating: test_data/SRR5946131.fastq (deflated 89%)
Archive:  project_data.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2026-02-17 03:22   example_data/
        0  2026-02-17 03:10   example_data/cancer_R1.fastq
   415650  2026-02-17 03:10   example_data/SRR5946129.fastq
  5525474  2026-02-17 03:10   example_data/SRR5946130.fastq
        0  2026-02-17 03:10   example_data/healthy_R2.fastq
        0  2026-02-17 03:10   example_data/cancer_R2.fastq
        0  2026-02-17 03:10   example_data/healthy_R1.fastq
        0  2026-02-17 03:22   test_data/
        0  2026-02-17 03:10   test_data/healthy_test_R1.fastq
 

In [45]:
!zip -r project_data.zip "/content/Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb" example_data/ test_data/


	zip warning: name not matched: /content/Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb
updating: example_data/ (stored 0%)
updating: example_data/SRR5946129.fastq (deflated 89%)
updating: example_data/SRR5946130.fastq (deflated 88%)
updating: test_data/ (stored 0%)
updating: test_data/SRR5946132.fastq (deflated 90%)
updating: test_data/SRR5946131.fastq (deflated 89%)


In [46]:
!ls /content/


example_data			fasterq.tmp.cb550c753c5b.36309	SRR2129993
fasterq.tmp.cb550c753c5b.11391	fasterq.tmp.cb550c753c5b.44125	SRR3179790
fasterq.tmp.cb550c753c5b.19812	fasterq.tmp.cb550c753c5b.4537	SRR33483210
fasterq.tmp.cb550c753c5b.22407	fasterq.tmp.cb550c753c5b.5951	SRR33483279
fasterq.tmp.cb550c753c5b.23535	project_data.zip		SRR5875050
fasterq.tmp.cb550c753c5b.27625	sample_data			test_data


In [47]:
!ls /content/


example_data			fasterq.tmp.cb550c753c5b.36309	SRR2129993
fasterq.tmp.cb550c753c5b.11391	fasterq.tmp.cb550c753c5b.44125	SRR3179790
fasterq.tmp.cb550c753c5b.19812	fasterq.tmp.cb550c753c5b.4537	SRR33483210
fasterq.tmp.cb550c753c5b.22407	fasterq.tmp.cb550c753c5b.5951	SRR33483279
fasterq.tmp.cb550c753c5b.23535	project_data.zip		SRR5875050
fasterq.tmp.cb550c753c5b.27625	sample_data			test_data


In [50]:
# Zip the notebook and the data folders together
!zip -r project_data.zip "/content/Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb" example_data test_data


	zip warning: name not matched: /content/Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb
updating: example_data/ (stored 0%)
updating: example_data/SRR5946129.fastq (deflated 89%)
updating: example_data/SRR5946130.fastq (deflated 88%)
updating: test_data/ (stored 0%)
updating: test_data/SRR5946132.fastq (deflated 90%)
updating: test_data/SRR5946131.fastq (deflated 89%)


In [51]:
!ls /content/


example_data			fasterq.tmp.cb550c753c5b.36309	SRR2129993
fasterq.tmp.cb550c753c5b.11391	fasterq.tmp.cb550c753c5b.44125	SRR3179790
fasterq.tmp.cb550c753c5b.19812	fasterq.tmp.cb550c753c5b.4537	SRR33483210
fasterq.tmp.cb550c753c5b.22407	fasterq.tmp.cb550c753c5b.5951	SRR33483279
fasterq.tmp.cb550c753c5b.23535	project_data.zip		SRR5875050
fasterq.tmp.cb550c753c5b.27625	sample_data			test_data


In [52]:
from google.colab import files
uploaded = files.upload()


Saving project_data.zip to project_data (1).zip


In [53]:
!zip -r project_data.zip "/content/Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb" example_data test_data


	zip warning: name not matched: /content/Copy of Molecular & Omics based Personalized Medicine task testing template.ipynb
updating: example_data/ (stored 0%)
updating: example_data/SRR5946129.fastq (deflated 89%)
updating: example_data/SRR5946130.fastq (deflated 88%)
updating: test_data/ (stored 0%)
updating: test_data/SRR5946132.fastq (deflated 90%)
updating: test_data/SRR5946131.fastq (deflated 89%)


In [ ]:
from google.colab import files
uploaded = files.upload()
